# Ingesting data into Amazon SageMaker FeatureStore

Kernel `Python 3 (Data Science)` works well with this notebook.

The following policies need to be attached to the execution role:
- AmazonSageMakerFullAccess
- AmazonS3FullAccess

## Contents
1. [Background](#Background)
1. [Setup SageMaker FeatureStore](#Setup-SageMaker-FeatureStore)
1. [Inspect Dataset](#Inspect-Dataset)
1. [Ingest Data into FeatureStore](#Ingest-Data-into-FeatureStore)

## Background

Amazon SageMaker FeatureStore is a new SageMaker capability that makes it easy for customers to create and manage curated data for machine learning (ML) development. SageMaker FeatureStore enables data ingestion via a high TPS API and data consumption via the online and offline stores. 

This notebook provides an example for the APIs provided by SageMaker FeatureStore by walking through the process of training a fraud detection model. The notebook demonstrates how the dataset's tables can be ingested into the FeatureStore, queried to create a training dataset, and quickly accessed during inference. 


### Terminology

A **FeatureGroup** is the main resource that contains the metadata for all the data stored in SageMaker FeatureStore. A FeatureGroup contains a list of FeatureDefinitions. A **FeatureDefinition** consists of a name and one of the following data types: a integral, string or decimal. The FeatureGroup also contains an **OnlineStoreConfig** and an **OfflineStoreConfig** controlling where the data is stored. Enabling the online store allows quick access to the latest value for a Record via the GetRecord API. The offline store, a required configuration, allows storage of historical data in your S3 bucket. 

Once a FeatureGroup is created, data can be added as Records. **Records** can be thought of as a row in a table. Each record will have a unique **RecordIdentifier** along with values for all other FeatureDefinitions in the FeatureGroup. 

## Setup SageMaker FeatureStore

Let's start by setting up the SageMaker Python SDK and boto client.

In [2]:
import boto3
import sagemaker
from sagemaker.session import Session


region = boto3.Session().region_name

boto_session = boto3.Session(region_name=region)

sagemaker_client = boto_session.client(service_name='sagemaker', region_name=region)
featurestore_runtime = boto_session.client(service_name='sagemaker-featurestore-runtime', region_name=region)

feature_store_session = Session(
    boto_session=boto_session,
    sagemaker_client=sagemaker_client,
    sagemaker_featurestore_runtime_client=featurestore_runtime
)

#### S3 Bucket Setup For The OfflineStore

SageMaker FeatureStore writes the data in the OfflineStore of a FeatureGroup to a S3 bucket owned by you. To be able to write to your S3 bucket, SageMaker FeatureStore assumes an IAM role which has access to it. The role is also owned by you.
Note that the same bucket can be re-used across FeatureGroups. Data in the bucket is partitioned by FeatureGroup.

Set the default s3 bucket name and it will be referenced throughout the notebook.

In [3]:
# You can modify the following to use a bucket of your choosing
default_s3_bucket_name = feature_store_session.default_bucket()
prefix = 'sagemaker-featurestore-demo'

print(default_s3_bucket_name)

sagemaker-us-east-1-155134682088


In [4]:
from sagemaker import get_execution_role

# You can modify the following to use a role of your choosing. See the documentation for how to create this.
role = get_execution_role()
print (role)

arn:aws:iam::155134682088:role/mod-6297809195fe4845-SageMakerExecutionRole-SWJFFCZXAUG9


## Load data

we will be using the data we stored in s3 during the previous notebook

In [5]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import io

s3_client = boto3.client('s3', region_name=region)

fraud_detection_bucket_name = default_s3_bucket_name
data_file_key = 'data/fraud-detection/credit-dataset.csv'

data_object = s3_client.get_object(Bucket=fraud_detection_bucket_name, Key=data_file_key)

transaction_data = pd.read_csv(io.BytesIO(data_object['Body'].read()))

transaction_data.head()


,time,v1,v2,v3,v4,v5,v6,v7,v8,v9,...,v23,v24,v25,v26,v27,v28,amount,class,event_time,record_id
0,157163.0,-0.181461,0.769439,2.228124,0.966996,-0.208874,0.190263,0.418665,-0.188676,-0.125505,...,-0.221410,-0.072678,-0.010541,-0.073326,-0.014187,-0.124904,19.99,0,2021-11-26 01:51:30.180052,255315
1,52321.0,0.068662,1.241962,-0.581812,0.235143,0.512704,-0.102508,-0.633509,-2.660118,-0.834111,...,-0.159685,-0.640054,0.693996,0.387699,0.060388,0.268200,0.76,0,2021-11-26 01:51:30.180052,67001
2,120318.0,2.031919,0.542767,-2.466979,1.248276,1.349961,-0.369090,0.599271,-0.197595,-0.212889,...,-0.061175,0.116552,0.537155,-0.513808,0.000423,-0.036472,1.00,0,2021-11-26 01:51:30.180052,170711
3,61694.0,-1.118980,-0.014418,1.003283,-1.388318,-0.564135,2.021013,0.974314,0.170433,0.685202,...,-0.316999,-1.614632,0.040428,0.876230,-0.262145,-0.022115,271.14,0,2021-11-26 01:51:30.180052,87463
4,144182.0,2.055312,-0.144976,-2.201962,-0.496593,0.008900,-2.564801,0.994471,-0.760230,0.378784,...,-0.089321,0.999290,0.469236,1.017879,-0.153530,-0.087704,37.50,0,2021-11-26 01:51:30.180052,225271


## Ingest Data into FeatureStore

In this step we will create the FeatureGroups representing the transaction and identity tables.

on a high level these are the steps to create a feature store.

* Create a feature group
* Create a feature group defnition (schema) and associate it to feature group
* Ingest data into the feauture group


In [6]:
from time import gmtime, strftime, sleep

fd_feature_group_name = 'transactionfeaturegroup0012'


In [7]:
from sagemaker.feature_store.feature_group import FeatureGroup

fd_feature_group = FeatureGroup(name=fd_feature_group_name, sagemaker_session=feature_store_session)

In [8]:
import time

current_time_sec = int(round(time.time()))

def cast_object_to_string(data_frame):
    for label in data_frame.columns:
        if data_frame.dtypes[label] == 'object':
            data_frame[label] = data_frame[label].astype("str").astype("string")

# cast object dtype to string. The SageMaker FeatureStore Python SDK will then map the string dtype to String feature type.
cast_object_to_string(transaction_data)

# record identifier and event time feature names
record_identifier_feature_name = "record_id"
event_time_feature_name = "event_time"

# append EventTime feature
transaction_data[event_time_feature_name] = pd.Series([current_time_sec]*len(transaction_data), dtype="float64")

# load feature definitions to the feature group. SageMaker FeatureStore Python SDK will auto-detect the data schema based on input data.
fd_feature_group.load_feature_definitions(data_frame=transaction_data); # output is suppressed


#### Create FeatureGroups in SageMaker FeatureStore

In [9]:
def wait_for_feature_group_creation_complete(feature_group):
    status = feature_group.describe().get("FeatureGroupStatus")
    while status == "Creating":
        print("Waiting for Feature Group Creation")
        time.sleep(5)
        status = feature_group.describe().get("FeatureGroupStatus")
    if status != "Created":
        raise RuntimeError(f"Failed to create feature group {feature_group.name}")
    print(f"FeatureGroup {feature_group.name} successfully created.")

fd_feature_group.create(
    s3_uri=f"s3://{default_s3_bucket_name}/{prefix}",
    record_identifier_name=record_identifier_feature_name,
    event_time_feature_name=event_time_feature_name,
    role_arn=role,
    enable_online_store=True,
  
)

wait_for_feature_group_creation_complete(feature_group=fd_feature_group)


Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
Waiting for Feature Group Creation
FeatureGroup transactionfeaturegroup0012 successfully created.


In [10]:
fd_feature_group.describe()

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:155134682088:feature-group/transactionfeaturegroup0012',
 'FeatureGroupName': 'transactionfeaturegroup0012',
 'RecordIdentifierFeatureName': 'record_id',
 'EventTimeFeatureName': 'event_time',
 'FeatureDefinitions': [{'FeatureName': 'time', 'FeatureType': 'Fractional'},
  {'FeatureName': 'v1', 'FeatureType': 'Fractional'},
  {'FeatureName': 'v2', 'FeatureType': 'Fractional'},
  {'FeatureName': 'v3', 'FeatureType': 'Fractional'},
  {'FeatureName': 'v4', 'FeatureType': 'Fractional'},
  {'FeatureName': 'v5', 'FeatureType': 'Fractional'},
  {'FeatureName': 'v6', 'FeatureType': 'Fractional'},
  {'FeatureName': 'v7', 'FeatureType': 'Fractional'},
  {'FeatureName': 'v8', 'FeatureType': 'Fractional'},
  {'FeatureName': 'v9', 'FeatureType': 'Fractional'},
  {'FeatureName': 'v10', 'FeatureType': 'Fractional'},
  {'FeatureName': 'v11', 'FeatureType': 'Fractional'},
  {'FeatureName': 'v12', 'FeatureType': 'Fractional'},
  {'FeatureName': 'v13', 'Fea

#### PutRecords into FeatureGroup

After the FeatureGroups have been created, we can put data into the FeatureGroups by using the PutRecord API. This API can handle high TPS and is designed to be called by different streams. The data from all of these Put requests is buffered and written to S3 in chunks. The files will be written to the offline store within a few minutes of ingestion. For this example, to accelerate the ingestion process, we are specifying multiple workers to do the job simultaneously. It will take ~5min to ingest data to the FeatureGroup, respectively.

In [ ]:
fd_feature_group.ingest(
    data_frame=transaction_data, max_workers=3, wait=False
)

To confirm that data has been ingested, we can quickly retrieve a record from the online store:

In [ ]:
record_identifier_value = str(100)

featurestore_runtime.get_record(FeatureGroupName=fd_feature_group_name, RecordIdentifierValueAsString=record_identifier_value)

In [30]:
%store fd_feature_group_name

Stored 'fd_feature_group_name' (str)
